In [1]:
################################
# install
################################
! pip install pandas
! pip install deep-translator
! pip install langdetect


In [5]:
################################
# imports
################################
import pandas as pd
import numpy as np
import os
import re
import nltk
from deep_translator import GoogleTranslator
from langdetect import detect
from tqdm import tqdm  #
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vanvi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vanvi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vanvi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vanvi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [3]:
################################
# test variable
################################

test = True

In [4]:
################################
# load in the world new data
################################

current_dir = os.getcwd()

if test:
    print('current_dir: ' + str(current_dir) + '\n')

#csv_path = current_dir + "\Top_25_World_News_2018_2023\WorldNewsData.csv"
csv_path = r"{}\Top_25_World_News_2018_2023\WorldNewsDataV2.csv".format(current_dir)

if test:
    print('csv_path: ' + str(csv_path) + '\n')

# convert csv to pandas dataframe
news_data = pd.read_csv(csv_path)

if test:
    print('len(news_data)    :' + str(len(news_data))    + '\n')
    print('news_data.iloc[0] :' + str(news_data.iloc[0]) + '\n')
    print('news_data.head()  :' + str(news_data.head())  + '\n')

current_dir: c:\Users\vanvi\Desktop\workspace\cmpe-256-project

csv_path: c:\Users\vanvi\Desktop\workspace\cmpe-256-project\Top_25_World_News_2018_2023\WorldNewsDataV2.csv

len(news_data)    :1859

news_data.iloc[0] :Date                                          May 01, 2018
Top1     North Korea to open its sky, South Korean medi...
Top2           The Mueller probe ain't ending anytime soon
Top3     BRAND NEW: 2018 – Renault Alpine A110 – Start-...
Top4                                           Loud Sound!
Top5     The story of the Midwestern innocent let loose...
Top6     Mattis tempers optimistic outlook on Afghanist...
Top7     Facebook is taking on Tinder with new dating f...
Top8     OECD chief says Greece deserves debt relief af...
Top9                     Facebook to launch dating service
Top10    U.N. could help Myanmar gather evidence of cri...
Top11    PTSD veterans 'cured' with ecstasy treatment i...
Top12    May Day protesters demand better rights for wo...
Top13    China’s

In [5]:
################################
# load in the stock data
################################

current_dir = os.getcwd()
dirs = [
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "forbes2000"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "nasdaq"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "nyse"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "sp500"),
       ]

#csv_files = [os.path.join(csv_dir, file) for file in os.listdir(csv_dir)]

if test:
    print('current_dir: ' + str(current_dir) + '\n')
    #print('csv_files  : ' + str(csv_files) + '\n')

# save list to handle duplicate stocks
processed_files = set()
stock_data      = []

# convert csv to pandas dataframe then combine
for dir in dirs:
    csv = [os.path.join(dir, file) for file in os.listdir(dir)]
    for file in csv:
        # take only first part of file name (leave out csv)
        file_name = os.path.splitext(os.path.basename(file))[0]
        # if duplicate, continue and skip
        if file_name in processed_files:
            continue
        # read in file and append
        df = pd.read_csv(file)
        df['Stocks'] = file_name
        stock_data.append(df)
        processed_files.add(file_name)

stock_data = pd.concat(stock_data, ignore_index=True)

if test:
    print('len(stock_data)    :' + str(len(stock_data))    + '\n')
    print('stock_data.iloc[0] :' + str(stock_data.iloc[0]) + '\n')
    print('stock_data.iloc[1] :' + str(stock_data.iloc[1]) + '\n')
    print('stock_data.head()  :' + str(stock_data.head()) + '\n')

current_dir: c:\Users\vanvi\Desktop\workspace\cmpe-256-project

len(stock_data)    :18988384

stock_data.iloc[0] :Date              18-11-1999
Low                28.612303
Open               32.546494
Volume            62546380.0
High               35.765381
Close              31.473534
Adjusted Close     27.066582
Stocks                     A
Name: 0, dtype: object

stock_data.iloc[1] :Date              19-11-1999
Low                28.478184
Open               30.713518
Volume            15234146.0
High               30.758226
Close              28.880545
Adjusted Close     24.836662
Stocks                     A
Name: 1, dtype: object

stock_data.head()  :         Date        Low       Open      Volume       High      Close  \
0  18-11-1999  28.612303  32.546494  62546380.0  35.765381  31.473534   
1  19-11-1999  28.478184  30.713518  15234146.0  30.758226  28.880545   
2  22-11-1999  28.657009  29.551144   6577870.0  31.473534  31.473534   
3  23-11-1999  28.612303  30.400572   5975

In [6]:
#######################################################
# remove dates older than May 01, 2018 for stock data
#######################################################

# stock_data.iloc[1] :Date              19-11-1999
stock_data['Date'] = pd.to_datetime(stock_data['Date'], format='%d-%m-%Y')
filt_new_data = stock_data[stock_data['Date'] >= '2018-05-01']
filt_new_data = filt_new_data.reset_index(drop=True)

if test:
    print('len(filt_new_data)    :' + str(len(filt_new_data))    + '\n')
    print('filt_new_data.head()  :' + str(filt_new_data.head())  + '\n')

len(filt_new_data)    :3930369

filt_new_data.head()  :        Date        Low       Open     Volume       High      Close  \
0 2018-05-01  65.489998  65.629997  3135100.0  66.349998  66.239998   
1 2018-05-02  65.769997  66.000000  2240500.0  66.860001  65.910004   
2 2018-05-03  64.860001  65.769997  2365900.0  66.459999  66.339996   
3 2018-05-04  65.540001  66.029999  1330800.0  67.250000  67.000000   
4 2018-05-07  67.070000  67.160004  1468700.0  67.980003  67.389999   

   Adjusted Close Stocks  
0       64.348511      A  
1       64.027931      A  
2       64.445656      A  
3       65.086807      A  
4       65.465668      A  



In [7]:
# convert news data format to stocks data

#news_data.iloc[0] :Date                                          May 01, 2018
news_data2 = news_data.copy()
#news_data2['Date'] = pd.to_datetime(news_data2['Date'], infer_datetime_format=True, errors='coerce')
#news_data2['Date Date'] = pd.to_datetime(news_data2['Date']).dt.strftime('%d-%m-%y')
news_data2['Date'] = pd.to_datetime(news_data2['Date'], format='%b %d, %Y', errors='coerce').dt.strftime('%d-%m-%y')

if test:
    print('len(news_data2)      :' + str(len(news_data2))      + '\n')
    print('news_data2.iloc[50] :' + str(news_data2.iloc[50])   + '\n')
    print('news_data2.head()    :' + str(news_data2.head())    + '\n')


len(news_data2)      :1859

news_data2.iloc[50] :Date                                              19-06-18
Top1     Bill to legalize recreational marijuana passes...
Top2     Canada just officially passed its law to legal...
Top3     Marijuana legalization Bill C-45 officially pa...
Top4     Senate passes pot bill, paving way for legal w...
Top5     Marijuana legalization Bill C-45 officially pa...
Top6                            Canada legalization of pot
Top7     US quits UN human rights council – 'a cesspool...
Top8     Tech condemns Trump: Apple, Microsoft, Airbnb ...
Top9     Senate passes pot bill, paving way for legal w...
Top10    Trump ‘loses temper’ and threatens government ...
Top11    Canadian Senate Passes Cannabis Legalization B...
Top12    Canada legalizing marijuana. Senate has voted ...
Top13    Trump was sustained through 2008 real estate c...
Top14    Canadian Senate passes pot bill, paving way fo...
Top15    Canadian Senate passes pot bill, paving way fo...
Top16  

In [8]:
news_data2.to_csv('updated_news_data.csv')
filt_new_data.to_csv('updated_stock_data.csv')

In [9]:
pip install deep-translator langdetect

In [10]:
def textTranslate(text):
    try:
        if pd.notna(text):  # Check if the text is not NaN
            lang = detect(text)
            if lang != "en":  # Translate only if not English
                return GoogleTranslator(source=lang, target='en').translate(text)
        return text  
    except:
        return text  

newsData = pd.read_csv('updated_news_data.csv')
columnsTranslate = newsData.columns[2:27]

for col in tqdm(columnsTranslate, desc='Translating Columns'):
    newsData[col] = newsData[col].astype(str).apply(textTranslate)

newsData.to_csv('translated_news_data.csv', index=False)
print(f'Translation complete and saved.')

Translating Columns: 100%|██████████| 25/25 [28:06<00:00, 67.44s/it]

Translation complete. Saved to translated_news_data.csv


In [11]:
# Next part, human eyes are needed in order to translate what is missed in the previous step.
# The new .csv file is called human_translated_news_data.csv

In [38]:
normalizedNewsData = pd.read_csv('human_translated_news_data.csv')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def textNormalization(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        tokens = word_tokenize(text)
        normalizedTokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return ' '.join(normalizedTokens)
    else:
        # Return empty string for non-string data
        return ''

# Normalize each headline in Top1 to Top25 columns
for col in normalizedNewsData.columns[2:]:
    normalizedNewsData[col] = normalizedNewsData[col].apply(textNormalization)

normalizedNewsData.to_csv('normalized_headlines.csv', index=False)
print(normalizedNewsData.head())  

   Unnamed: 0      Date                                               Top1  \
0           0  1-May-18    north korea open sky south korean medium report   
1           1  2-May-18  got fear trump threatens declassify fbi justic...   
2           2  3-May-18  hollywood isnt simply churning crummy remake n...   
3           3  4-May-18  donald trump say london hospital like war zone...   
4           4  5-May-18  missing teen mom implores canada take note u s...   

                                                Top2  \
0             mueller probe aint ending anytime soon   
1  best garage heater review incuding electric ga...   
2  iran foreign minister u consistently violated ...   
3  german seat un security council likely israel ...   
4      william kate release new picture prince louis   

                                                Top3  \
0  brand new 2018renault alpine a110startup amp l...   
1  apple fueled everyones biggest fear company go...   
2  rihanna cover june 2018

In [56]:
def replaceDuplicates(data):
    seen = set()
    duplicatePositions = []

    for col in df.columns[2:]:
        for index, value in data[col].items():
            if pd.notna(value):
                if value in seen:
                    data.at[index, col] = 'nan'
                    duplicatePositions.append((index, col))
                else:
                    seen.add(value)
    return data, duplicatePositions

cleanedData, duplicatePositions = replaceDuplicates(normalizedNewsData.copy())
cleanedData.to_csv('cleaned_headlines.csv', index=False)
print(f'Total duplicates: {len(duplicatePositions)}')
print(f'\nHere are all of the duplicate news titles: {duplicatePositions}')

Total duplicates: 4795

Here are all of the duplicate news titles: [(27, 'Top1'), (32, 'Top1'), (55, 'Top1'), (63, 'Top1'), (67, 'Top1'), (102, 'Top1'), (106, 'Top1'), (154, 'Top1'), (171, 'Top1'), (172, 'Top1'), (183, 'Top1'), (216, 'Top1'), (245, 'Top1'), (246, 'Top1'), (328, 'Top1'), (346, 'Top1'), (353, 'Top1'), (385, 'Top1'), (422, 'Top1'), (521, 'Top1'), (529, 'Top1'), (541, 'Top1'), (546, 'Top1'), (566, 'Top1'), (575, 'Top1'), (576, 'Top1'), (698, 'Top1'), (716, 'Top1'), (718, 'Top1'), (770, 'Top1'), (777, 'Top1'), (850, 'Top1'), (856, 'Top1'), (869, 'Top1'), (917, 'Top1'), (918, 'Top1'), (919, 'Top1'), (920, 'Top1'), (921, 'Top1'), (922, 'Top1'), (923, 'Top1'), (924, 'Top1'), (925, 'Top1'), (926, 'Top1'), (927, 'Top1'), (928, 'Top1'), (929, 'Top1'), (930, 'Top1'), (931, 'Top1'), (932, 'Top1'), (933, 'Top1'), (934, 'Top1'), (935, 'Top1'), (936, 'Top1'), (937, 'Top1'), (938, 'Top1'), (939, 'Top1'), (940, 'Top1'), (941, 'Top1'), (942, 'Top1'), (943, 'Top1'), (944, 'Top1'), (945, '